In [1]:
import pandas as pd
import joblib
import os

In [4]:
data = pd.read_pickle('Output/test.pkl')
data.shape

(924621, 1103)

In [5]:
def predict(data):
    
    models = [
        'fold_0_iter',
        'fold_2_iter',
        'fold_4_iter',

        'fold_1_0_iter',
        'fold_1_3_iter',
        'fold_1_4_iter',
        'fold_3_0_iter',
        'fold_3_1_iter',
        'fold_3_3_iter',

        'weak_fold_'
    ]

    model_list = []
    for path in ['Models_DART_L1,L2/', 'Models_DART_L3/']:
        for fname in os.listdir(path):
            for model_name in models:
                if model_name in fname:
                    model_list.append(path + fname)

    pred_list = []
    for model_path in model_list:
        print(model_path)
        print('{:.02f} MB'.format(os.path.getsize(model_path)/1000000))
        model = joblib.load(model_path)
        pred_list.append(model.predict(data))
    
    return pred_list

In [6]:
pred_list = predict(data)

Models_DART_L1,L2/fold_2_iter_15673_score_0.79786.pkl
192.18 MB
Models_DART_L1,L2/fold_3_3_iter_12338_score_0.79752.pkl
141.22 MB
Models_DART_L1,L2/fold_3_1_iter_9891_score_0.79668.pkl
113.30 MB
Models_DART_L1,L2/fold_1_4_iter_6205_score_0.80461.pkl
71.20 MB
Models_DART_L1,L2/fold_3_0_iter_12713_score_0.79723.pkl
148.29 MB
Models_DART_L1,L2/fold_0_iter_14937_score_0.80361.pkl
171.27 MB
Models_DART_L1,L2/fold_1_3_iter_8889_score_0.80380.pkl
101.84 MB
Models_DART_L1,L2/fold_4_iter_13953_score_0.79769.pkl
153.02 MB
Models_DART_L1,L2/fold_1_0_iter_6194_score_0.79579.pkl
71.04 MB
Models_DART_L3/weak_fold_3_score_0.79656.pkl
176.48 MB
Models_DART_L3/weak_fold_7_score_0.78550.pkl
109.77 MB
Models_DART_L3/weak_fold_8_score_0.79930.pkl
218.46 MB
Models_DART_L3/weak_fold_6_score_0.80330.pkl
229.89 MB
Models_DART_L3/weak_fold_4_score_0.79648.pkl
288.36 MB
Models_DART_L3/weak_fold_0_score_0.79444.pkl
313.61 MB
Models_DART_L3/weak_fold_9_score_0.78429.pkl
89.47 MB
Models_DART_L3/weak_fold_5_score_0

In [7]:
column_name = [
    'fold_2',
    'fold_3_3',
    'fold_3_1',
    'fold_1_4',
    'fold_3_0',
    'fold_0',
    'fold_1_3',
    'fold_4',
    'fold_1_0',
    'weak_fold_3',
    'weak_fold_7',
    'weak_fold_8',
    'weak_fold_6',
    'weak_fold_4',
    'weak_fold_0',
    'weak_fold_9',
    'weak_fold_5',
    'weak_fold_1',
    'weak_fold_2',
]

pred_df = pd.DataFrame(pred_list).T
pred_df.columns = column_name
pred_df.index = data.index

In [8]:
pred_df.to_csv('Output/test_DART_folds.csv')